In [1]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

import pickle

# Import dataset

In [2]:
data = pd.read_csv("../DataFormating/final.csv")

In [3]:
data.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year,home_rank,home_total_points,home_previous_points,home_rank_change,home_cur_year_avg,...,away_cur_year_avg,away_cur_year_avg_weighted,away_last_year_avg,away_last_year_avg_weighted,away_two_year_ago_avg,away_two_year_ago_weighted,away_three_year_ago_avg,away_three_year_ago_weighted,Home Avg Goals,Away Avg Goals
0,1.0,Mexico,4.0,France,1930.0,12,0.0,45,7,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,Spain,0.0,Portugal,1930.0,23,0.0,38,10,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,Poland,0.0,Sweden,1930.0,4,0.0,55,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,Sweden,2.0,Belgium,1930.0,18,0.0,47,-1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,Germany,6.0,Denmark,1930.0,10,0.0,51,-1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Set `X` and `y`

In [4]:
data.columns

Index(['Away Team Goals', 'Away Team Name', 'Home Team Goals',
       'Home Team Name', 'Year', 'home_rank', 'home_total_points',
       'home_previous_points', 'home_rank_change', 'home_cur_year_avg',
       'home_cur_year_avg_weighted', 'home_last_year_avg',
       'home_last_year_avg_weighted', 'home_two_year_ago_avg',
       'home_two_year_ago_weighted', 'home_three_year_ago_avg',
       'home_three_year_ago_weighted', 'away_rank', 'away_total_points',
       'away_previous_points', 'away_rank_change', 'away_cur_year_avg',
       'away_cur_year_avg_weighted', 'away_last_year_avg',
       'away_last_year_avg_weighted', 'away_two_year_ago_avg',
       'away_two_year_ago_weighted', 'away_three_year_ago_avg',
       'away_three_year_ago_weighted', 'Home Avg Goals', 'Away Avg Goals'],
      dtype='object')

In [5]:
X = data.drop(["Away Team Goals", "Home Team Goals"], axis=1)

y = []
for i in range(len(data)):
    home_team_goals = data.iloc[i]["Home Team Goals"]
    away_team_goals = data.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [6]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [7]:
word_cup_teams = [
    "Egypt",
    "Morocco",
    "Nigeria",
    "Senegal",
    "Tunisia",
    "Australia",
    "IR Iran",
    "Japan",
    "Korea DPR",
    "Saudi Arabia",
    "Belgium",
    "Croatia",
    "Denmark",
    "England",
    "France",
    "Germany",
    "Iceland",
    "Poland",
    "Portugal",
    "Russia",
    "Serbia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Costa Rica",
    "Mexico",
    "Panama",
    "Argentina",
    "Brazil",
    "Colombia",
    "Peru",
    "Uruguay"
]

team_names = list(data["Home Team Name"].unique()) + list(data["Away Team Name"].unique()) + word_cup_teams

In [8]:
team_name_encoder = LabelEncoder().fit(team_names)

In [9]:
X["Home Team Name"] = team_name_encoder.transform(X["Home Team Name"])
X["Away Team Name"] = team_name_encoder.transform(X["Away Team Name"])

### Feature Selection

In [10]:
feature_names = [
    "Team Name",
    "rank",
    "points",
    "Avg Goals"
]

COLUMNS = []
for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

X = X[COLUMNS]

In [11]:
COLUMNS

['Away Team Name',
 'Home Team Name',
 'home_rank',
 'home_total_points',
 'home_previous_points',
 'home_rank_change',
 'away_rank',
 'away_total_points',
 'away_previous_points',
 'away_rank_change',
 'Home Avg Goals',
 'Away Avg Goals']

# Traning Session

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 12 columns):
Away Team Name          3951 non-null int64
Home Team Name          3951 non-null int64
home_rank               3951 non-null int64
home_total_points       3951 non-null float64
home_previous_points    3951 non-null int64
home_rank_change        3951 non-null int64
away_rank               3951 non-null int64
away_total_points       3951 non-null float64
away_previous_points    3951 non-null int64
away_rank_change        3951 non-null int64
Home Avg Goals          3951 non-null float64
Away Avg Goals          3951 non-null float64
dtypes: float64(4), int64(8)
memory usage: 370.5 KB


In [13]:
X.describe()

,Away Team Name,Home Team Name,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Avg Goals,Away Avg Goals
count,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000,3951.000000
mean,15.941028,14.582890,21.247785,109.117747,314.709694,1.379904,22.881296,108.048502,308.869906,1.212098,1.448520,1.024867
std,10.281891,10.158125,19.831125,318.596757,402.940843,5.371249,21.330939,317.040579,400.626058,5.223808,1.245904,0.960996
min,0.000000,0.000000,1.000000,0.000000,0.000000,-18.000000,1.000000,0.000000,0.000000,-20.000000,0.000000,0.000000
25%,7.000000,5.000000,7.000000,0.000000,45.500000,-1.000000,7.000000,0.000000,45.000000,-1.000000,0.500000,0.000000
50%,16.000000,13.000000,14.000000,0.000000,55.000000,0.000000,17.000000,0.000000,55.000000,0.000000,1.333333,1.000000
75%,25.000000,23.000000,31.000000,0.000000,648.000000,4.000000,32.000000,0.000000,640.000000,3.000000,2.080000,1.500000
max,32.000000,32.000000,139.000000,1725.290000,1887.000000,23.000000,139.000000,1725.290000,1887.000000,23.000000,9.000000,10.000000


In [14]:
xgb_model = XGBClassifier(
    n_estimators=500,
    max_depth=3,
    learning_rate=0.03,
    booster="gbtree",
    n_jobs=-1
)

In [15]:
xgb_model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# Save model and encoders

In [16]:
with open("xgb_model.b", "wb") as f:
    pickle.dump(xgb_model, f)
    
with open("team_name_encoder.b", "wb") as f:
    pickle.dump(team_name_encoder, f)